In [1]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownTextSplitter
from langchain.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import tqdm
import os

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [49]:
from langchain.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import WebBaseLoader

loader = PyPDFLoader("docs/AT_Internal rules for interns_VN.pdf")
# loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

pages = loader.load()

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)


In [50]:

text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

In [51]:
docs = text_splitter.split_documents(pages)

In [52]:
model = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceEmbeddings(model_name = model)

d:\thực tập doanh nghiệp\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [53]:
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_index")


In [7]:
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [54]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [57]:
question = 'hãy nêu thời gian làm việc của công ty'
docs = vectorstore.similarity_search(question,k=5)
print(docs[0].page_content)
print()
print(docs[1].page_content)
print()
print(docs[2].page_content)

phụ trách để xin phép thay đổi thời gian thực tập. • Mọi vắng mặt không lý do, hoặc không tuân thủ thời gian làm việc của công ty với lý do không hợp lý đều sẽ ảnh hướng đến kết quả đánh giá thực tập sinh vào cuối đợt thực tập.  2. Một số quy định chung trong giờ làm việc • Ăn mặc gọn gàng, lịch sự, phù hợp môi trường văn phòng. • Không làm việc riêng trong giờ làm việc (xem phim, chat, Facebook … những nội dung không phục vụ cho công việc đang làm).. • Không download dữ liệu cá nhân (phim, ảnh, nhạc) trong và ngoài giờ làm việc. • Không chơi games trong và ngoài giờ làm việc trừ các dịp đặc biệt được ban giám đốc chấp thuận. • Điện thoại để ở chế độ rung • Thân thiện với người trong công ty, tác phong nhanh nhẹn, chuyên nghiệp, tôn trọng trong công việc. • Sử dụng tiết kiệm các loại văn phòng phẩm, điện, tắt màn hình trước khi rời chỗ làm việc • Tuyệt đối bảo mật thông tin (xem rõ phụ lục đính kèm Nội quy công ty này) 3. Một số quy định riêng về thái độ làm việc • Thực tập sinh cần

p

In [9]:
key = ''
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=key

genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [60]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [61]:
response = model.generate_content("What is the meaning of life?")
to_markdown(response.text)

> As a large language model, I don't have personal beliefs or experiences to draw on.  The meaning of life is a question philosophers have pondered for centuries, and there's no single right answer. It's a deeply personal and individual question. 
> 
> However, I can offer some perspectives:
> 
> * **It's up to you to decide:**  Many believe that the meaning of life isn't inherent but rather something we create through our experiences, relationships, and contributions. 
> * **Finding purpose:** Some find meaning in pursuing a specific purpose, whether it's a career, creative passion, religious devotion, or helping others.
> * **Living in the moment:** Others find meaning in appreciating the present moment and finding joy in everyday experiences.
> * **Connecting with something larger:** Some people find meaning in connecting with something beyond themselves, such as nature, spirituality, or a sense of community.
> 
> Ultimately, the most important thing is to find what gives your life meaning and purpose. 
> 
> If you're interested in exploring this further, you might enjoy reading philosophical works, exploring different spiritual traditions, or engaging in meaningful conversations with others. 


In [62]:


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_response(input_text):
    doc = format_docs(vectorstore.similarity_search(input_text,k=5))
#Hãy trả lời không biết nêu không có thông tin tron đoạn ngữ cảnh
    prompt_text = f"""
        Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi.
        Hãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh
        

        đoạn ngữ cảnh: {doc}

        câu hỏi: {input_text}

        câu trả lời:
        """


    respone = model.generate_content(prompt_text)
    return to_markdown(respone.text).data


In [63]:
import gradio as gr


iface = gr.Interface(generate_response, 
                     inputs="textbox",
                     outputs="textbox",
                     title="Chatbot about Deep learning",
                     description="Enter your message and the chatbot will respond accordingly.")
iface.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
